In [3]:
from gurobipy import *
import numpy as np
import time

In [4]:
#Abrir un archivo excel
import pandas as pd

#como leer un archivo .csv
data = pd.read_csv('clientes2.csv', sep=';')
print(data)


            x         y   demanda
0      7,9744  -11,3045   147,199
1    -13,6786   -3,3825   161,327
2     -8,8032  -10,3279  172,4834
3    -15,3621    9,4144  133,8201
4     -9,0891   -7,9311  129,8221
..        ...       ...       ...
995   -6,2143    -10,86  155,4059
996  -11,7203   -2,1472  132,1921
997    6,4149    7,9825  126,1607
998    5,9054   12,7097  156,3924
999   -9,4583   -7,1544  141,3835

[1000 rows x 3 columns]


## Pre-procesamiento

In [5]:
for i in range(len(data)):
    for j in range(len(data.columns)):
        valor = data.iloc[i,j]
        valor = valor.replace(",",".")
        data.iloc[i,j] = valor
data = data.applymap(float)

data = data.iloc[0:100]
print(data)

          x        y   demanda
0    7.9744 -11.3045  147.1990
1  -13.6786  -3.3825  161.3270
2   -8.8032 -10.3279  172.4834
3  -15.3621   9.4144  133.8201
4   -9.0891  -7.9311  129.8221
..      ...      ...       ...
95  11.0695  -2.9302  135.5393
96  16.9123 -15.1201  138.4456
97   5.5644  11.8088  150.9736
98  11.3843 -11.6451  145.2344
99 -14.3093  -8.0562  186.7664

[100 rows x 3 columns]


C:\Users\mrive\AppData\Local\Temp\ipykernel_22452\3244050827.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(float)


In [18]:
import random
import time

def centro_euclidiano(x=0, y=0, data=0):
    
    x_i = data.iloc[:,0]
    y_i = data.iloc[:,1]
    pesos = data.iloc[:,2]
    sum = 0
    sum_new_x = 0
    sum_g_i = 0
    sum_new_y = 0
    for i in range(len(data)):
    
        distancia = ((x-float(x_i[i]))**2+(y-float(y_i[i]))**2)**(1/2)
        fx = float(pesos[i])*distancia
        sum += fx
        
        g_i= (pesos[i]/distancia)
        
        sum_new_x += (x_i[i]*g_i)
        sum_new_y += (y_i[i]*g_i)
        sum_g_i += g_i
        
    x_new = sum_new_x/sum_g_i
    y_new = sum_new_y/sum_g_i
    
    return x_new, y_new
        
x=random.randint(-2,2)
y=random.randint(-2,2)
x_new, y_new = random.randint(3,5), random.randint(3,5)
contador = 0
tiempo_0 = time.time()

while (abs(x-x_new)>0.00001 and abs(y-y_new)>0.00001):
    contador += 1
    x, y = x_new, y_new
    x_new, y_new = centro_euclidiano(x, y, data)
    
    # Logs
    # print(f'La distancia entre x: {x} y x_new: {x_new} es: {abs(x-x_new)}')
    # print(f'La distancia entre y: {y} y y_new: {y_new} es: {abs(y-y_new)}')
    
    tiempo_n = time.time()
    
print(f'El modelo de localización arrojó las coordenadas x: {x_new} y: {y_new}')
print(f'El modelo se demoró un tiempo de {tiempo_n-tiempo_0} segundos y {contador} iteraciones')
    

El modelo de localización arrojó las coordenadas x: -3.9863734097722876 y: -5.046098594419144
El modelo se demoró un tiempo de 0.024375438690185547 segundos y 21 iteraciones
